In [1]:

import pandas as pd
import numpy as np
import matplotlib as plt
import os
import sys
import json
import csv


In [2]:
# os.getcwd()
def read_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Error: File not found at path {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Unable to decode JSON from file {file_path}")
        return None

In [3]:
overview = read_json_file("overview_IBM.json")

In [9]:
import re

daily_stock_prices = read_json_file("daily_adjusted_full_IBM.json")

dividends = read_json_file("cash_flow_IBM.json")["quarterlyReports"]
dividends

# {dividends[0]['fiscalDateEnding']:dividends[0]['dividendPayout']}

[{'fiscalDateEnding': '2023-09-30',
  'reportedCurrency': 'USD',
  'operatingCashflow': '3056000000',
  'paymentsForOperatingActivities': 'None',
  'proceedsFromOperatingActivities': 'None',
  'changeInOperatingLiabilities': 'None',
  'changeInOperatingAssets': 'None',
  'depreciationDepletionAndAmortization': '1093000000',
  'capitalExpenditures': '281000000',
  'changeInReceivables': 'None',
  'changeInInventory': 'None',
  'profitLoss': '1714000000',
  'cashflowFromInvestment': '-1953000000',
  'cashflowFromFinancing': '-3132000000',
  'proceedsFromRepaymentsOfShortTermDebt': '9000000',
  'paymentsForRepurchaseOfCommonStock': 'None',
  'paymentsForRepurchaseOfEquity': 'None',
  'paymentsForRepurchaseOfPreferredStock': 'None',
  'dividendPayout': '1515000000',
  'dividendPayoutCommonStock': '1515000000',
  'dividendPayoutPreferredStock': 'None',
  'proceedsFromIssuanceOfCommonStock': 'None',
  'proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet': '154000000',
  'proceedsFromIs

In [10]:
dividends =  [ (x['fiscalDateEnding'],float(x['dividendPayout'] )) for x in dividends]
dividends

[('2023-09-30', 1515000000.0),
 ('2023-06-30', 1510000000.0),
 ('2023-03-31', 1497000000.0),
 ('2022-12-31', 1494000000.0),
 ('2022-09-30', 1491000000.0),
 ('2022-06-30', 1488000000.0),
 ('2022-03-31', 1475000000.0),
 ('2021-12-31', 1474000000.0),
 ('2021-09-30', 1471000000.0),
 ('2021-06-30', 1467000000.0),
 ('2021-03-31', 1457000000.0),
 ('2020-12-31', 1454000000.0),
 ('2020-09-30', 1453000000.0),
 ('2020-06-30', 1450000000.0),
 ('2020-03-31', 1440000000.0),
 ('2019-12-31', 1438000000.0),
 ('2019-09-30', 1436000000.0),
 ('2019-06-30', 1436000000.0),
 ('2019-03-31', 1397000000.0),
 ('2018-12-31', 1416000000.0)]

In [6]:
beta = float(overview["Beta"])
roe = float(overview["ReturnOnEquityTTM"])
eps = float(overview["EPS"])
divps = float(overview["DividendPerShare"])
payout_ratio = divps/eps
div_growth_rate = roe * (1 - payout_ratio)
market_return = 0.11
prev_close = float(daily_stock_prices['Time Series (Daily)'][next(iter(daily_stock_prices['Time Series (Daily)']))].get('4. close'))

risk_free_rate = prev_close * 0.01

cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)
#############






In [ ]:
from pyspark import SparkContext, SparkConf
from datetime import datetime, timedelta

# Initialize Spark
conf = SparkConf().setAppName("Example").setMaster("local")
sc = SparkContext(conf=conf)

In [17]:
# Function to calculate present value (PV) factor
def calculate_pv_factor(row):
    fiscal_date_ending = datetime.strptime(row[0], "%Y-%m-%d")
    pv_factor = (fiscal_date_ending + timedelta(days=365) - datetime.now()).days / 365.0
    return (row[0], row[1], pv_factor)

rdd = sc.parallelize(dividends)

# Apply the transformation to the RDD
transformed_rdd = rdd.map(calculate_pv_factor)

# Function to calculate dividend growth
def calculate_dividend_growth(row):
    if row[1] != 0:
        dividend_growth = row[1] * (1 + 0.46045880000000006) / ((1 + row[2]) ** 2)
        return (row[0], row[1], row[2], dividend_growth)
    else:
        return (row[0], row[1], row[2], 0.0)

# Apply the transformation to the transformed RDD
dividend_growth = transformed_rdd.map(calculate_dividend_growth)

In [18]:
# Filter out zero dividends
filtered_rdd = transformed_rdd.filter(lambda row: row[1] != 0)
# Collect and print the result
# result_list = result_rdd.collect()
# for row in result_list:
#     print(row)

In [20]:
# Calculate present value of dividends
pv_divs = filtered_rdd.map(lambda row: row[1] / (1 + cost_of_equity) ** row[2]).sum()


In [29]:
# Calculate terminal value
last_row = filtered_rdd.reduce(lambda x, y: y)

last_row[2]

-3.96986301369863

In [30]:
terminal_value = (last_row[2] * (1 + div_growth_rate) / (cost_of_equity - div_growth_rate)) / (1 + cost_of_equity) ** last_row[2]

{"pv_divs": pv_divs, "terminal_value": terminal_value}

{'pv_divs': 61240588122.84652, 'terminal_value': -45.34180401051845}

In [ ]:
result = multi_ddm(spark, ticker='swk', discount_rate=cost_of_equity, div_growth_rate=div_growth_rate)
print(result)

In [35]:
from pyspark import SparkContext,SparkSession
from datetime import datetime, timedelta

daily_stock_prices = read_json_file("daily_adjusted_full_IBM.json")

dividends_raw = read_json_file("cash_flow_IBM.json")["quarterlyReports"]

dividends =  [ (x['fiscalDateEnding'],float(x['dividendPayout'] )) for x in dividends_raw]

div_df = pd.DataFrame(dividends)
dividends_estimate = div_df[1].mode()*4

beta = float(overview["Beta"])
roe = float(overview["ReturnOnEquityTTM"])
eps = float(overview["EPS"])
divps = float(overview["DividendPerShare"])
payout_ratio = divps/eps
div_growth_rate = roe * (1 - payout_ratio)
market_return = 0.11
prev_close = float(daily_stock_prices['Time Series (Daily)'][next(iter(daily_stock_prices['Time Series (Daily)']))].get('4. close'))

risk_free_rate = prev_close * 0.01

cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)

def calculate_pv_factor(row):
    fiscal_date_ending = datetime.strptime(row[0], "%Y-%m-%d")
    pv_factor = (fiscal_date_ending + timedelta(days=365) - datetime.now()).days / 365.0
    return (row[0], row[1], pv_factor)

def calculate_dividend_growth(row, growth_rate):
    if row[1] != 0:
        dividend_growth = row[1] * (1 + growth_rate * row[2])
        return (row[0], row[1], row[2], dividend_growth)
    else:
        return (row[0], row[1], row[2], 0.0)

def multi_ddm(row):
  '''
  Model used to predict the value of a stock given future dividend payments.
  
  discount_rate can be set as the cost of equity.

  div_growth_rate standard input is a company's ROE multiplied by the company's retention ratio.

  Time frames: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max. **Note that if the time frame is changed from the default then the
  num_future_dividends parameter also needs to be changed. For exampe if time_frame = '2y', then set the num_future_dividends to 8.

  growth_rate is an increase in the dividend amount distributed.
  '''
  time_frame='1y'
  num_future_dividends = 4
  growth_rate=3
  ticker = yf.Ticker(row[0])
  beta = ticker.info['beta']
  risk_free_rate = ticker.get_info()['previousClose']*.01
  market_return = .11

  discount_rate = risk_free_rate + beta *(market_return-risk_free_rate)


  div_growth_rate = stanley_bd.info['returnOnEquity'] * (1-stanley_bd.get_info()['payoutRatio'])

  div_per_share = stanley_bd.get_info()['previousClose'] * stanley_bd.info['dividendYield']
    
  dividend_history = ticker.history(period=time_frame)['Dividends'][ticker.history(period=time_frame)['Dividends'] != 0]
  dividend_estimate = [dividend_history.mode().iloc[-1]]*num_future_dividends
  pv_factor = [( (date + datetime.timedelta(days=365)).to_pydatetime(pytz.UTC) - datetime.datetime.now(pytz.UTC)).days/365 for date in  dividend_history.index]
  dividend_growth = [dividend * 1+(growth_rate*factor) for dividend,factor in zip(dividend_estimate,pv_factor)]
  pv_divs = sum([dividend/(1+discount_rate)**factor for factor,dividend in zip(pv_factor,dividend_growth)])
  terminal_value = (dividend_growth[-1] * (1+div_growth_rate)/(discount_rate - div_growth_rate))/(1+discount_rate)**pv_factor[-1]
  ddm = pv_divs+terminal_value

  return (row[0],ddm)



# Example usage
if __name__ == "__main__":
    # Initialize SparkContext

    sc = SparkContext(appName="DDMVModel")
    

    # Sample data for demonstration
    data = dividends
    # Example parameters
    discount_rate = cost_of_equity
    growth_rate = 0.0

    # Calculate DDM value using RDD
    ddm_result = multi_ddm_rdd(sc, data, discount_rate, div_growth_rate, growth_rate)

    print("DDM Value:")
    print(ddm_result)

    # Stop the SparkContext
    sc.stop()


DDM Value:
77413436973.63766


In [ ]:
import requests
import csv
import pandas as pd
from pyspark import SparkContext

CSV_URL = 'https://www.alphavantage.co/query?function=LISTING_STATUS&status=active&apikey=SBDZS38E0OSX9D8S'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)

    stock_symbols_df = pd.DataFrame(my_list[1:],columns=my_list[0])
    # for row in my_list:
    #     print(row)
stocks_df = stock_symbols_df[stock_symbols_df['assetType']=='Stock']['symbol']

sc = SparkContext(appName="DDMVModel")
sc_df = sc.parallelize(stocks_df)

In [33]:

import yfinance as yf

def multi_ddm(row):
  '''
  Model used to predict the value of a stock given future dividend payments.
  
  discount_rate can be set as the cost of equity.

  div_growth_rate standard input is a company's ROE multiplied by the company's retention ratio.

  Time frames: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max. **Note that if the time frame is changed from the default then the
  num_future_dividends parameter also needs to be changed. For exampe if time_frame = '2y', then set the num_future_dividends to 8.

  growth_rate is an increase in the dividend amount distributed.
  '''
  time_frame='1y'
  num_future_dividends = 4
  growth_rate=3
  ticker = yf.Ticker(row[0])
  beta = ticker.info['beta']
  risk_free_rate = ticker.get_info()['previousClose']*.01
  market_return = .11

  discount_rate = risk_free_rate + beta *(market_return-risk_free_rate)


  div_growth_rate = ticker.info['returnOnEquity'] * (1-ticker.get_info()['payoutRatio'])

  div_per_share = ticker.get_info()['previousClose'] * ticker.info['dividendYield']
    
  dividend_history = ticker.history(period=time_frame)['Dividends'][ticker.history(period=time_frame)['Dividends'] != 0]
  dividend_estimate = [dividend_history.mode().iloc[-1]]*num_future_dividends
  pv_factor = [( (date + datetime.timedelta(days=365)).to_pydatetime(pytz.UTC) - datetime.datetime.now(pytz.UTC)).days/365 for date in  dividend_history.index]
  dividend_growth = [dividend * 1+(growth_rate*factor) for dividend,factor in zip(dividend_estimate,pv_factor)]
  pv_divs = sum([dividend/(1+discount_rate)**factor for factor,dividend in zip(pv_factor,dividend_growth)])
  terminal_value = (dividend_growth[-1] * (1+div_growth_rate)/(discount_rate - div_growth_rate))/(1+discount_rate)**pv_factor[-1]
  ddm = pv_divs+terminal_value

  return (row[0],ddm)

ddm_df = sc_df.map(multi_ddm)

